In [3]:
import numpy as np
from tensorly.decomposition import tucker
import tensorly as tl

In [146]:
def make_tensor_decomposition(
    l_matrix, u_block_size=2,
    q_block_size=2,
    i_core=2,
    j_core=2,
    k_core=2
):
    l_matrix_shape = l_matrix.shape[0]
    u_matrix = (
        l_matrix
            .reshape((l_matrix_shape, l_matrix_shape))
            .reshape(
                l_matrix_shape // u_block_size,
                u_block_size,
                l_matrix_shape // u_block_size,
                u_block_size
            ).mean(axis=(1, 3))
    )
    n = (l_matrix_shape // u_block_size // q_block_size) ** 2
    block_matrix = (
        u_matrix
            .reshape(
                u_matrix.shape[0] // q_block_size,
                q_block_size, -1, q_block_size
            ).swapaxes(1, 2)
            .reshape(-1, q_block_size, q_block_size)
    )
    x = tl.tensor(block_matrix, dtype=tl.float32)
    return tucker(x, ranks=[i_core, j_core, k_core])


def make_hash(core, a_factor, b_factor, c_factor):
    a_p = a_factor.mean(axis=1)
    a_h = (a_p >= a_p.mean()).astype(int)
    b_p = b_factor.mean(axis=1)
    b_h = (b_p >= b_p.mean()).astype(int)
    c_p = c_factor.mean(axis=1)
    c_h = (c_p >= c_p.mean()).astype(int)
    
    return np.hstack((a_h, b_h, c_h))
    
    

In [176]:
sh = 256
a = make_tensor(
    np.random.normal(20, size=(sh,sh)),
    q_block_size=32
)

In [175]:
make_hash(a[0], *a[1])

array([0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])